# Imports

In [ ]:
# If you need some install, uncomment the code bellow 

# !pip install torch
# !pip install torchvision
# !pip install torchaudio
# !pip install ipdb

In [10]:
import torchaudio
import torch
from matplotlib import pyplot as plt
import numpy as np
from torch import Tensor
import os
from typing import Tuple

import ipdb

# from torchaudio.datasets import YESNO, LIBRISPEECH
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import pdb, traceback, sys


from torchaudio.datasets.utils import (
  download_url,
  extract_archive,
  walk_files,
)

# MaseseSpeech 2h

In [11]:
# # Train link Drive
# https://drive.google.com/file/d/1CfqHBiOEVnJiuwZoqBJMlGO-N97JD3sR/view?usp=sharing
    
# wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1CfqHBiOEVnJiuwZoqBJMlGO-N97JD3sR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1CfqHBiOEVnJiuwZoqBJMlGO-N97JD3sR" -O train-clean.tar.xz && rm -rf /tmp/cookies.txt
    
# # Valid link Drive
# https://drive.google.com/file/d/1Y1CiB7TbrGdghVokwMTBQA1fRza2NZUP/view?usp=sharing
# https://drive.google.com/file/d/1Y1CiB7TbrGdghVokwMTBQA1fRza2NZUP/view?usp=sharing
    
# wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Y1CiB7TbrGdghVokwMTBQA1fRza2NZUP' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Y1CiB7TbrGdghVokwMTBQA1fRza2NZUP" -O dev-clean.tar.xz && rm -rf /tmp/cookies.txt

In [12]:
# !mkdir MaseseSpeech
# !mkdir MaseseSpeech

# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1CfqHBiOEVnJiuwZoqBJMlGO-N97JD3sR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1CfqHBiOEVnJiuwZoqBJMlGO-N97JD3sR" -O MaseseSpeech/train-clean.tar.xz && rm -rf /tmp/cookies.txt
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Y1CiB7TbrGdghVokwMTBQA1fRza2NZUP' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Y1CiB7TbrGdghVokwMTBQA1fRza2NZUP" -O MaseseSpeech/dev-clean.tar.xz && rm -rf /tmp/cookies.txt

# extract_archive("MaseseSpeech/train-clean.tar.xz")
# extract_archive("MaseseSpeech/dev-clean.tar.xz")

# !rm -r "MaseseSpeech/train-clean.tar.xz"
# !rm -r "MaseseSpeech/dev-clean.tar.xz"

In [13]:


URL = "train-clean"
FOLDER_IN_ARCHIVE = "MaseseSpeech"
# BASE_URL = "https://dl.fbaipublicfiles.com/librispeech_100h_mp3/"
# _CHECKSUMS = {
#   BASE_URL + "dev-clean.tar.gz":
#   "076916a8f9c61951c5d2e6efaa8d2188232fcf860eec8c074e46edf4fac9623e",
#   BASE_URL + "test-clean.tar.gz":
#   "3c171e2f1e377e4993c2dbe6bff3f01cd324c0ed462f4de6c78737402a7dbedd",
#   BASE_URL + "train-clean-100.tar.gz":
#   "7bfbefc680d25ba3a82798ce32c287ea0e82932af1b1f864fae71fb52d2f41f0",
# }


def load_masesespeech_item(fileid: str, 
                          path: str, 
                          ext_audio: str, 
                          ext_txt: str) -> Tuple[Tensor, int, str, int, int, int]:
    
    speaker_id, chapter_id, utterance_id = fileid.split("-")
    
    file_text = speaker_id + "-" + chapter_id + ext_txt
    file_text = os.path.join(path, speaker_id, chapter_id, file_text)
    
    fileid_audio = speaker_id + "-" + chapter_id + "-" + utterance_id
    file_audio = fileid_audio + ext_audio
    file_audio = os.path.join(path, speaker_id, chapter_id, file_audio)
    
    
    

    try :
        # Load audio
        waveform, sample_rate = torchaudio.load(file_audio)

        # Load text
        with open(file_text) as ft:
            for line in ft:
                fileid_text, utterance = line.strip().split(" ", 1)
                if fileid_audio == fileid_text:
                    break
            else:
              # Translation not found
              raise FileNotFoundError("Translation not found for " + fileid_audio)
    except:
        print(file_audio)
        print(waveform)
        pass
#         traceback.print_exc()
                
    return (
        waveform,
        sample_rate,
        utterance,
#         int(speaker_id),
        int(chapter_id),
        int(utterance_id)
        )


class MASESESPEECH_2H_MP3(Dataset):
    """
    Create a Dataset for MaseseSpeech. Each item is a tuple of the form:
    waveform, utterance, chapter_id, verse_id, utterance_id
    """
    
    _ext_txt = ".trans.txt"
    _ext_audio = ".wav"
    
    def __init__(self,
                 root: str,
                 mode: str = "MaseseSpeech/train-clean",
                 folder_in_archive: str = FOLDER_IN_ARCHIVE,
#                  download: bool = False
                ) -> None:
#         if url in [
#             "dev-clean",
# #             "test-clean",
#             "train-clean",]:
            
#             ext_archive = ".tar.xz"
#             base_url = BASE_URL
#             url = os.path.join(base_url, url + ext_archive)

#         basename = os.path.basename(url)
#         archive = os.path.join(root, basename)

#         basename = basename.split(".")[0]
#         folder_in_archive = os.path.join(folder_in_archive, basename)
        
        
        self._path = mode

#         if download:
#             if not os.path.isdir(self._path):
#                 if not os.path.isfile(archive):
#                     checksum = _CHECKSUMS.get(url, None)
#                     download_url(url, root, hash_value=checksum)
        
        walker = walk_files(
          self._path, suffix=self._ext_audio, prefix=False, remove_suffix=True
        )
        self._walker = list(walker)
    def __getitem__(self, n: int) -> Tuple[Tensor, int, str, int, int, int]:
        fileid = self._walker[n]
        return load_masesespeech_item(fileid, self._path, self._ext_audio, self._ext_txt)
    
    def __len__(self) -> int:
        return len(self._walker)

In [74]:
masese_train = MASESESPEECH_2H_MP3(".", mode = "MaseseSpeech/train-clean")
masese_dev = MASESESPEECH_2H_MP3(".", mode = "MaseseSpeech/dev-clean")

In [75]:
# just so you get an idea of the format 
print(next(iter(masese_train)))
print(next(iter(masese_dev)))

(tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.0923, -0.0903, -0.0939]]), 16000, 'NA LOFUNDO, MOTO ABIMISAKA KAKA ETUMBA,  KASI BWANYA EZALI EPAI YA BAOYO BAPESANAKA MAKANISI.', 13, 9)
(tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0609, 0.0486, 0.0518]]), 16000, 'MPO LITOMBA YA KOZWA YANGO ELEKI LITOMBA YA KOZWA PALATA MPE KOZWA YANGO ELEKI KOZWA WOLO.', 3, 13)


In [76]:
# Use this is your acoustic model is outputting letters
special_caracters = "Ɛ,;.ƆÁÓ-?:*É—“”!Í(10)"
tokens_list = list(" ABCDEFGHIJKLMNOPQRSTUVWXYZ"+special_caracters)
tokens_set = set(tokens_list)
print(tokens_list)

[' ', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'Ɛ', ',', ';', '.', 'Ɔ', 'Á', 'Ó', '-', '?', ':', '*', 'É', '—', '“', '”', '!', 'Í', '(', '1', '0', ')']


In [77]:
def collate_fn_libri(batch):
    #print(batch)
    tensors = [b[0].t() for b in batch if b]
    tensors_len = [len(t) for t in tensors]
    tensors = torch.nn.utils.rnn.pad_sequence(tensors, batch_first=True)
    tensors = tensors.transpose(1, -1)
    
    transcriptions = [list(b[2].replace("'", " ")) for b in batch if b]
    targets = [torch.tensor([tokens_list.index(e) for e in t]) for t in transcriptions]
    targets_len = [len(t) for t in targets]
    targets = torch.nn.utils.rnn.pad_sequence(targets, batch_first=True)
    
    return tensors, targets, torch.tensor(tensors_len), torch.tensor(targets_len)

# train_set = torch.utils.data.DataLoader(masese_train, batch_size=900, shuffle=True,
#                                         num_workers=4, collate_fn=collate_fn_libri)

In [78]:
# torchaudio.load("MaseseSpeech/train-clean/020/008/020-008-017.wav")

In [79]:
# print(next(iter(train_set)))
train_set = torch.utils.data.DataLoader(masese_train, batch_size=64, shuffle=True,
                                        num_workers=2, collate_fn=collate_fn_libri)

In [80]:
def collate_fn_libri(batch):
    #print(batch)
    tensors = [b[0].t() for b in batch if b]
    tensors_len = [len(t) for t in tensors]
    tensors = torch.nn.utils.rnn.pad_sequence(tensors, batch_first=True)
    tensors = tensors.transpose(1, -1)
    
    transcriptions = [list(b[2].replace("'", " ")) for b in batch if b]
    targets = [torch.tensor([tokens_list.index(e) for e in t]) for t in transcriptions]
    targets_len = [len(t) for t in targets]
    targets = torch.nn.utils.rnn.pad_sequence(targets, batch_first=True)
    
    return tensors, targets, torch.tensor(tensors_len), torch.tensor(targets_len)

valid_set = torch.utils.data.DataLoader(masese_dev, batch_size=1500, shuffle=True,
                                        num_workers=4, collate_fn=collate_fn_libri)

In [94]:
print(next(iter(valid_set)))


(tensor([[[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]]), tensor([[11, 15, 19,  ...,  0,  0,  0],
        [14,  1,  0,  ...,  0,  0,  0],
        [14,  1,  0,  ...,  0,  0,  0],
        ...,
        [ 2, 15, 12,  ...,  0,  0,  0],
        [13, 16, 15,  ...,  0,  0,  0],
        [13, 23,  1,  ...,  0,  0,  0]]), tensor([183315, 163350, 350658, 288585, 224697, 193479, 188760, 169884, 157542,
        253737, 162261, 116160, 176418, 235950, 142296, 120153, 185493, 293304,
        132495, 233046, 238150, 135762, 186219, 182952, 184404, 225060, 186582,
        223245, 267894, 204732, 249381, 161535, 173151, 246114, 217437, 263538,
        178959, 202917, 198924, 224697, 207636, 223971, 197109, 146289, 280236,
        117975, 217437, 191301, 170610, 221793, 104181, 172062, 233046,